# 在 Google Colab 中进行 EchoHeart 的 Qwen 微调

这个 Notebook 会自动配置环境并启动使用 Qwen 对 EchoHeart 模型进行微调的过程。

**步骤：**
1. 克隆 GitHub 仓库。
2. 安装必要的依赖项。
3. 运行训练脚本。

In [ ]:
# 1. 克隆仓库
!git clone https://github.com/shuakami/echoheart_demo.git
%cd echoheart_demo

In [ ]:
# 2. 安装依赖
!pip install -q -r requirements.txt

In [ ]:
# 3. 运行训练脚本
!python train.py

## 训练完成！

微调后的模型保存在 Colab 环境文件系统的 `output/qwen-ft` 目录中。